In [1]:
import cv2
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers.legacy import Adam

In [2]:
c = 0
for files in os.listdir("./train"):
    c += 1
print(c)

25000


In [3]:
data = []
for files in os.listdir('./train'):
    image = cv2.resize(cv2.imread(os.path.join('./train', files)), (100, 100))
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    label = files.split('.')[0]
    if label == 'cat':
        data.append([gray, np.array([0, 1])])
    elif label == 'dog':
        data.append([gray, np.array([1, 0])])

In [4]:
len(data)

25000

In [5]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(100, 100, 1), padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.summary()

Metal device set to: Apple M1 Pro


2023-02-18 10:52:50.968050: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-18 10:52:50.968253: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 64)      640       
                                                                 
 conv2d_1 (Conv2D)           (None, 98, 98, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 47, 47, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 21, 21, 32)        1

In [6]:
train , test = train_test_split(data, test_size = 0.1, shuffle = True)

In [7]:
print(len(train))
print(len(test))

22500
2500


In [8]:
X = np.array([i[0] for i in train]).reshape(-1, 100, 100, 1)
Y = np.array([i[1] for i in train])

x_valid = np.array([i[0] for i in test]).reshape(-1, 100, 100, 1)
y_valid = np.array([i[1] for i in test])

In [9]:
X = X.astype('float32')/255
x_valid = x_valid.astype('float32')/255

In [10]:
print(X.shape)
print(Y.shape)
print(x_valid.shape)
print(y_valid.shape)

(22500, 100, 100, 1)
(22500, 2)
(2500, 100, 100, 1)
(2500, 2)


In [15]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss= 'categorical_crossentropy', optimizer= opt, metrics=['accuracy'])

In [16]:
model.fit(X, Y, batch_size= 10, epochs= 3, verbose= 1, validation_data=(x_valid,y_valid))

Epoch 1/3


2023-02-18 11:01:50.805388: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-18 11:01:51.078634: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2250/2250 [==============================] - ETA: 0s - loss: 0.6686 - accuracy: 0.5734

2023-02-18 11:02:38.361016: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2250/2250 [==============================] - 50s 22ms/step - loss: 0.6686 - accuracy: 0.5734 - val_loss: 0.6388 - val_accuracy: 0.6304
Epoch 2/3
2250/2250 [==============================] - 49s 22ms/step - loss: 0.5989 - accuracy: 0.6781 - val_loss: 0.5575 - val_accuracy: 0.7132
Epoch 3/3
2250/2250 [==============================] - 49s 22ms/step - loss: 0.5192 - accuracy: 0.7412 - val_loss: 0.4904 - val_accuracy: 0.7616


In [32]:
model.save('/Users/mm22/Documents/Development/sample_catdog_identification')

INFO:tensorflow:Assets written to: /Users/mm22/Documents/Development/sample_catdog_identification/assets


INFO:tensorflow:Assets written to: /Users/mm22/Documents/Development/sample_catdog_identification/assets


In [31]:
CATEGORIES = ['Cat', 'Dog']

def prepare(filepath):
    IMG_SIZE = 100
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

prediction = model.predict([prepare('test1/79.jpg')])
#print(CATEGORIES[int(prediction[0][0])])
print(prediction[0][0])

1/1 [==============================] - 0s 13ms/step
1.0
